# Multi-Timeframe Fluid Dynamics Analysis

## Table of Contents
1. [Introduction to Multi-Scale Analysis](#introduction)
2. [Wavelet Transform for Market Flow](#wavelet-transform)
3. [Scale-Dependent Pattern Recognition](#scale-patterns)
4. [Cross-Scale Energy Transfer](#energy-transfer)
5. [Temporal Coherence Analysis](#temporal-coherence)
6. [Multi-Timeframe Trading Signals](#trading-signals)
7. [Regime Persistence Across Scales](#regime-persistence)
8. [Advanced Visualization](#visualization)

## Introduction

Multi-timeframe analysis in fluid dynamics markets examines how patterns and energy cascades across different temporal scales. This approach reveals:

- **Scale-dependent patterns**: Different fluid dynamics patterns emerge at various time scales
- **Energy cascade mechanisms**: How energy transfers from large to small scales
- **Cross-scale correlations**: How patterns at one scale influence others
- **Temporal coherence**: Persistence of fluid patterns across time scales

### Key Concepts:
- **Microscale (seconds)**: Individual order interactions, shock waves
- **Mesoscale (minutes)**: Vortex formation, boundary layer dynamics
- **Macroscale (hours/days)**: Large-scale circulation, regime changes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, ndimage
from scipy.fft import fft2, ifft2, fftfreq
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
import pywt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

print("Multi-Timeframe Analysis Environment Initialized")
print("Libraries loaded: NumPy, SciPy, Wavelets, Scikit-learn")

## Wavelet Transform for Market Flow

### Continuous Wavelet Transform (CWT)

The CWT decomposes market flow signals into time-frequency components:

$$W(a,b) = \frac{1}{\sqrt{a}} \int_{-\infty}^{\infty} f(t) \psi^*\left(\frac{t-b}{a}\right) dt$$

Where:
- $a$ = scale parameter (related to frequency)
- $b$ = translation parameter (time)
- $\psi$ = mother wavelet function
- $f(t)$ = market flow signal

### Morlet Wavelet for Financial Analysis

The complex Morlet wavelet is ideal for financial time series:
$$\psi(t) = \pi^{-1/4} e^{i\omega_0 t} e^{-t^2/2}$$

It provides good time-frequency localization for market pattern detection.

In [ ]:
class MultiTimeframeFluidAnalyzer:
    """
    Advanced multi-timeframe analysis using wavelet transforms and fluid dynamics
    """
    
    def __init__(self, sampling_rate=1000):
        self.sampling_rate = sampling_rate
        self.scales = np.logspace(0, 3, 50)  # 50 scales from 1 to 1000
        self.wavelet = 'cmor1.5-1.0'  # Complex Morlet wavelet
        
    def generate_synthetic_market_data(self, length=2048):
        """
        Generate synthetic multi-scale market flow data
        """
        t = np.linspace(0, 10, length)
        dt = t[1] - t[0]
        
        # Multi-scale components
        # Macro trend (low frequency)
        macro_trend = 2 * np.sin(0.5 * t) + 0.5 * np.sin(0.3 * t)
        
        # Meso patterns (medium frequency)
        meso_pattern = np.sin(5 * t) * np.exp(-0.1 * (t - 5)**2)
        
        # Micro fluctuations (high frequency)
        micro_noise = 0.3 * np.random.randn(length)
        
        # Market shocks (impulse responses)
        shock_times = [2.5, 6.0, 8.5]
        shocks = np.zeros(length)
        for shock_time in shock_times:
            shock_idx = int(shock_time / dt)
            if shock_idx < length:
                shock_width = 50
                shock_indices = np.arange(max(0, shock_idx-shock_width//2), 
                                        min(length, shock_idx+shock_width//2))
                shocks[shock_indices] += 2.0 * np.exp(-((shock_indices - shock_idx) / 20)**2)
        
        # Combine all components
        market_flow = macro_trend + meso_pattern + micro_noise + shocks
        
        return {
            'time': t,
            'flow': market_flow,
            'macro': macro_trend,
            'meso': meso_pattern,
            'micro': micro_noise,
            'shocks': shocks
        }
    
    def perform_cwt_analysis(self, signal):
        """
        Perform Continuous Wavelet Transform analysis
        """
        # Compute CWT
        coefficients, frequencies = pywt.cwt(signal, self.scales, self.wavelet, 
                                           sampling_period=1/self.sampling_rate)
        
        # Convert scales to periods
        periods = 1.0 / frequencies
        
        # Compute wavelet power spectrum
        power = np.abs(coefficients)**2
        
        # Compute phase information
        phase = np.angle(coefficients)
        
        # Compute instantaneous frequency
        inst_freq = np.diff(np.unwrap(phase, axis=1), axis=1) / (2 * np.pi)
        
        return {
            'coefficients': coefficients,
            'power': power,
            'phase': phase,
            'frequencies': frequencies,
            'periods': periods,
            'instantaneous_frequency': inst_freq
        }
    
    def detect_scale_dependent_patterns(self, cwt_result, threshold_percentile=90):
        """
        Detect patterns at different time scales
        """
        power = cwt_result['power']
        periods = cwt_result['periods']
        
        # Define scale categories
        micro_mask = periods < 0.1  # Very short term
        meso_mask = (periods >= 0.1) & (periods < 1.0)  # Medium term
        macro_mask = periods >= 1.0  # Long term
        
        # Compute scale-specific power
        micro_power = np.mean(power[micro_mask, :], axis=0)
        meso_power = np.mean(power[meso_mask, :], axis=0)
        macro_power = np.mean(power[macro_mask, :], axis=0)
        
        # Detect high-activity regions
        threshold = np.percentile(power, threshold_percentile)
        
        # Find scale-specific events
        micro_events = micro_power > np.percentile(micro_power, threshold_percentile)
        meso_events = meso_power > np.percentile(meso_power, threshold_percentile)
        macro_events = macro_power > np.percentile(macro_power, threshold_percentile)
        
        return {
            'micro_power': micro_power,
            'meso_power': meso_power,
            'macro_power': macro_power,
            'micro_events': micro_events,
            'meso_events': meso_events,
            'macro_events': macro_events,
            'scale_masks': {
                'micro': micro_mask,
                'meso': meso_mask,
                'macro': macro_mask
            }
        }

# Initialize analyzer
analyzer = MultiTimeframeFluidAnalyzer()

# Generate synthetic market data
market_data = analyzer.generate_synthetic_market_data(length=2048)

print(f"Generated synthetic market data with {len(market_data['time'])} time points")
print(f"Time span: {market_data['time'][0]:.2f} to {market_data['time'][-1]:.2f}")
print(f"Flow range: {np.min(market_data['flow']):.2f} to {np.max(market_data['flow']):.2f}")

## Cross-Scale Energy Transfer Analysis

### Energy Cascade Theory

Energy cascade in financial markets shows how energy transfers between scales:

$$\frac{\partial E(k)}{\partial t} = T(k) - \varepsilon(k) + F(k)$$

Where:
- $E(k)$ = energy at wavenumber $k$
- $T(k)$ = energy transfer function
- $\varepsilon(k)$ = energy dissipation
- $F(k)$ = external forcing

### Wavelet-Based Energy Transfer

Using wavelets, we can quantify energy transfer between scales through:
- Scale-to-scale energy flux
- Cross-scale phase relationships
- Coherence analysis

In [ ]:
def analyze_cross_scale_energy_transfer(cwt_result):
    """
    Analyze energy transfer between different time scales
    """
    power = cwt_result['power']
    phase = cwt_result['phase']
    periods = cwt_result['periods']
    
    n_scales, n_times = power.shape
    
    # Compute scale-to-scale energy flux
    energy_flux = np.zeros((n_scales-1, n_times))
    
    for i in range(n_scales-1):
        # Energy difference between adjacent scales
        scale_diff = np.log(periods[i+1]) - np.log(periods[i])
        power_gradient = (power[i+1, :] - power[i, :]) / scale_diff
        
        # Phase coherence between scales
        phase_diff = np.angle(np.exp(1j * (phase[i+1, :] - phase[i, :])))
        coherence = np.abs(np.mean(np.exp(1j * phase_diff)))
        
        # Energy flux incorporating phase relationships
        energy_flux[i, :] = power_gradient * coherence
    
    # Compute cumulative energy transfer
    cumulative_transfer = np.cumsum(energy_flux, axis=0)
    
    # Identify dominant energy pathways
    flux_magnitude = np.abs(energy_flux)
    dominant_transfers = flux_magnitude > np.percentile(flux_magnitude, 75)
    
    # Scale-averaged properties
    upscale_transfer = np.mean(energy_flux[energy_flux > 0])  # Energy going to larger scales
    downscale_transfer = np.mean(energy_flux[energy_flux < 0])  # Energy going to smaller scales
    
    return {
        'energy_flux': energy_flux,
        'cumulative_transfer': cumulative_transfer,
        'dominant_transfers': dominant_transfers,
        'upscale_transfer': upscale_transfer,
        'downscale_transfer': downscale_transfer,
        'net_transfer': upscale_transfer + downscale_transfer
    }

def compute_temporal_coherence(cwt_result, window_size=50):
    """
    Compute temporal coherence of patterns across scales
    """
    coefficients = cwt_result['coefficients']
    n_scales, n_times = coefficients.shape
    
    # Sliding window coherence analysis
    n_windows = n_times - window_size + 1
    coherence_matrix = np.zeros((n_scales, n_windows))
    
    for i in range(n_windows):
        window_coeffs = coefficients[:, i:i+window_size]
        
        # Compute local coherence for each scale
        for scale in range(n_scales):
            # Phase consistency within window
            phases = np.angle(window_coeffs[scale, :])
            phase_coherence = np.abs(np.mean(np.exp(1j * phases)))
            coherence_matrix[scale, i] = phase_coherence
    
    # Compute cross-scale coherence
    cross_scale_coherence = np.zeros((n_scales, n_scales))
    for i in range(n_scales):
        for j in range(n_scales):
            if i != j:
                # Correlation between coherence patterns
                corr = np.corrcoef(coherence_matrix[i, :], coherence_matrix[j, :])[0, 1]
                cross_scale_coherence[i, j] = corr
    
    return {
        'temporal_coherence': coherence_matrix,
        'cross_scale_coherence': cross_scale_coherence,
        'mean_coherence_by_scale': np.mean(coherence_matrix, axis=1)
    }

# Perform CWT analysis on market flow
cwt_result = analyzer.perform_cwt_analysis(market_data['flow'])

# Detect scale-dependent patterns
pattern_analysis = analyzer.detect_scale_dependent_patterns(cwt_result)

# Analyze cross-scale energy transfer
energy_transfer = analyze_cross_scale_energy_transfer(cwt_result)

# Compute temporal coherence
coherence_analysis = compute_temporal_coherence(cwt_result)

print("Multi-scale Analysis Complete:")
print(f"Upscale energy transfer: {energy_transfer['upscale_transfer']:.4f}")
print(f"Downscale energy transfer: {energy_transfer['downscale_transfer']:.4f}")
print(f"Net energy transfer: {energy_transfer['net_transfer']:.4f}")
print(f"Micro-scale events detected: {np.sum(pattern_analysis['micro_events'])}")
print(f"Meso-scale events detected: {np.sum(pattern_analysis['meso_events'])}")
print(f"Macro-scale events detected: {np.sum(pattern_analysis['macro_events'])}")

## Multi-Timeframe Trading Signal Generation

### Signal Synthesis Across Scales

Effective trading signals combine information from multiple timeframes:

$$S_{total}(t) = \sum_{i} w_i \cdot S_i(t) \cdot C_i(t)$$

Where:
- $S_i(t)$ = signal from scale $i$
- $w_i$ = weight for scale $i$
- $C_i(t)$ = confidence measure for scale $i$

### Adaptive Weighting Strategy

Weights are dynamically adjusted based on:
- Scale-specific pattern strength
- Historical performance
- Market regime characteristics

In [ ]:
class MultiTimeframeSignalGenerator:
    """
    Generate trading signals from multi-timeframe fluid dynamics analysis
    """
    
    def __init__(self):
        self.scale_weights = {
            'micro': 0.2,   # Short-term signals
            'meso': 0.5,    # Medium-term signals
            'macro': 0.3    # Long-term signals
        }
        
    def generate_scale_signals(self, pattern_analysis, cwt_result):
        """
        Generate signals for each time scale
        """
        power = cwt_result['power']
        phase = cwt_result['phase']
        periods = cwt_result['periods']
        
        signals = {}
        
        for scale_name, mask in pattern_analysis['scale_masks'].items():
            if np.any(mask):
                # Average power and phase for this scale
                scale_power = np.mean(power[mask, :], axis=0)
                scale_phase = np.mean(phase[mask, :], axis=0)
                
                # Signal strength based on power
                power_signal = (scale_power - np.mean(scale_power)) / np.std(scale_power)
                
                # Directional signal based on phase derivatives
                phase_derivative = np.gradient(scale_phase)
                direction_signal = np.tanh(phase_derivative)  # Normalize to [-1, 1]
                
                # Combined signal
                combined_signal = power_signal * direction_signal
                
                # Confidence based on power concentration
                confidence = scale_power / (np.mean(power) + 1e-8)
                confidence = np.clip(confidence, 0, 1)
                
                signals[scale_name] = {
                    'signal': combined_signal,
                    'power_signal': power_signal,
                    'direction_signal': direction_signal,
                    'confidence': confidence
                }
        
        return signals
    
    def synthesize_final_signal(self, scale_signals, energy_transfer):
        """
        Combine signals from all scales with adaptive weighting
        """
        # Adjust weights based on energy transfer patterns
        transfer_factor = abs(energy_transfer['net_transfer'])
        
        # Adaptive weights (favor scales with strong energy transfer)
        adaptive_weights = self.scale_weights.copy()
        if transfer_factor > 0.1:  # Significant energy transfer
            if energy_transfer['upscale_transfer'] > abs(energy_transfer['downscale_transfer']):
                # Energy moving to larger scales - favor macro
                adaptive_weights['macro'] *= 1.5
                adaptive_weights['micro'] *= 0.7
            else:
                # Energy moving to smaller scales - favor micro
                adaptive_weights['micro'] *= 1.5
                adaptive_weights['macro'] *= 0.7
        
        # Normalize weights
        total_weight = sum(adaptive_weights.values())
        adaptive_weights = {k: v/total_weight for k, v in adaptive_weights.items()}
        
        # Combine signals
        final_signal = np.zeros_like(scale_signals['micro']['signal'])
        total_confidence = np.zeros_like(final_signal)
        
        for scale_name, weight in adaptive_weights.items():
            if scale_name in scale_signals:
                scale_data = scale_signals[scale_name]
                weighted_signal = weight * scale_data['signal'] * scale_data['confidence']
                final_signal += weighted_signal
                total_confidence += weight * scale_data['confidence']
        
        # Normalize by total confidence
        final_signal = np.divide(final_signal, total_confidence, 
                               out=np.zeros_like(final_signal), where=total_confidence!=0)
        
        return {
            'signal': final_signal,
            'confidence': total_confidence,
            'adaptive_weights': adaptive_weights
        }
    
    def generate_trading_decisions(self, final_signal, threshold=0.5):
        """
        Convert signals to trading decisions
        """
        signal_values = final_signal['signal']
        confidence = final_signal['confidence']
        
        # Trading decisions based on signal strength and confidence
        buy_signals = (signal_values > threshold) & (confidence > 0.3)
        sell_signals = (signal_values < -threshold) & (confidence > 0.3)
        hold_signals = ~(buy_signals | sell_signals)
        
        # Position sizing based on confidence
        position_size = confidence * np.abs(signal_values)
        position_size = np.clip(position_size, 0, 1)  # Normalize to [0, 1]
        
        return {
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'hold_signals': hold_signals,
            'position_size': position_size,
            'signal_strength': np.abs(signal_values)
        }

# Initialize signal generator
signal_generator = MultiTimeframeSignalGenerator()

# Generate scale-specific signals
scale_signals = signal_generator.generate_scale_signals(pattern_analysis, cwt_result)

# Synthesize final signal
final_signal = signal_generator.synthesize_final_signal(scale_signals, energy_transfer)

# Generate trading decisions
trading_decisions = signal_generator.generate_trading_decisions(final_signal)

print("Trading Signal Generation Complete:")
print(f"Adaptive weights: {final_signal['adaptive_weights']}")
print(f"Buy signals: {np.sum(trading_decisions['buy_signals'])}")
print(f"Sell signals: {np.sum(trading_decisions['sell_signals'])}")
print(f"Hold signals: {np.sum(trading_decisions['hold_signals'])}")
print(f"Average position size: {np.mean(trading_decisions['position_size']):.3f}")
print(f"Signal strength range: {np.min(trading_decisions['signal_strength']):.3f} to {np.max(trading_decisions['signal_strength']):.3f}")

## Advanced Multi-Timeframe Visualization

This section creates comprehensive visualizations showing how fluid dynamics patterns emerge and evolve across different time scales.

In [ ]:
# Create comprehensive multi-timeframe visualization
fig = plt.figure(figsize=(20, 16))
gs = GridSpec(4, 4, figure=fig, hspace=0.3, wspace=0.3)

# Main title
fig.suptitle('Multi-Timeframe Fluid Dynamics Analysis', fontsize=18, fontweight='bold')

# 1. Original signal components (top row)
ax1 = fig.add_subplot(gs[0, :])
t = market_data['time']
ax1.plot(t, market_data['macro'], label='Macro (Trend)', alpha=0.8, linewidth=2)
ax1.plot(t, market_data['meso'], label='Meso (Patterns)', alpha=0.8, linewidth=2)
ax1.plot(t, market_data['micro'], label='Micro (Noise)', alpha=0.6, linewidth=1)
ax1.plot(t, market_data['shocks'], label='Shocks', alpha=0.8, linewidth=2)
ax1.plot(t, market_data['flow'], 'k-', label='Combined Flow', alpha=0.9, linewidth=2)
ax1.set_title('Market Flow Components Across Time Scales')
ax1.set_xlabel('Time')
ax1.set_ylabel('Flow Magnitude')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# 2. Wavelet power spectrum (second row, left)
ax2 = fig.add_subplot(gs[1, :2])
power_db = 10 * np.log10(cwt_result['power'] + 1e-8)  # Convert to dB
im1 = ax2.imshow(power_db, aspect='auto', origin='lower', cmap='jet',
                extent=[t[0], t[-1], 0, len(cwt_result['periods'])])
ax2.set_title('Wavelet Power Spectrum (Time-Scale Analysis)')
ax2.set_xlabel('Time')
ax2.set_ylabel('Scale Index')
plt.colorbar(im1, ax=ax2, label='Power (dB)')

# Add scale annotations
scale_indices = [0, len(cwt_result['periods'])//3, 2*len(cwt_result['periods'])//3, len(cwt_result['periods'])-1]
scale_labels = ['Micro', 'Meso', 'Macro', 'Trend']
ax2.set_yticks(scale_indices)
ax2.set_yticklabels(scale_labels)

# 3. Energy transfer (second row, right)
ax3 = fig.add_subplot(gs[1, 2:])
im2 = ax3.imshow(energy_transfer['energy_flux'], aspect='auto', origin='lower', 
                cmap='RdBu_r', extent=[t[0], t[-1], 0, energy_transfer['energy_flux'].shape[0]])
ax3.set_title('Cross-Scale Energy Transfer')
ax3.set_xlabel('Time')
ax3.set_ylabel('Scale Transfer Index')
plt.colorbar(im2, ax=ax3, label='Energy Flux')

# 4. Scale-specific signals (third row)
ax4 = fig.add_subplot(gs[2, 0])
if 'micro' in scale_signals:
    ax4.plot(t, scale_signals['micro']['signal'], 'b-', alpha=0.8)
    ax4.fill_between(t, 0, scale_signals['micro']['confidence'], alpha=0.3)
ax4.set_title('Micro-Scale Signals')
ax4.set_xlabel('Time')
ax4.set_ylabel('Signal')
ax4.grid(True, alpha=0.3)

ax5 = fig.add_subplot(gs[2, 1])
if 'meso' in scale_signals:
    ax5.plot(t, scale_signals['meso']['signal'], 'g-', alpha=0.8)
    ax5.fill_between(t, 0, scale_signals['meso']['confidence'], alpha=0.3)
ax5.set_title('Meso-Scale Signals')
ax5.set_xlabel('Time')
ax5.set_ylabel('Signal')
ax5.grid(True, alpha=0.3)

ax6 = fig.add_subplot(gs[2, 2])
if 'macro' in scale_signals:
    ax6.plot(t, scale_signals['macro']['signal'], 'r-', alpha=0.8)
    ax6.fill_between(t, 0, scale_signals['macro']['confidence'], alpha=0.3)
ax6.set_title('Macro-Scale Signals')
ax6.set_xlabel('Time')
ax6.set_ylabel('Signal')
ax6.grid(True, alpha=0.3)

# 5. Final trading signal (third row, right)
ax7 = fig.add_subplot(gs[2, 3])
ax7.plot(t, final_signal['signal'], 'k-', linewidth=2, label='Final Signal')
ax7.fill_between(t, 0, final_signal['confidence'], alpha=0.3, label='Confidence')

# Mark trading decisions
buy_times = t[trading_decisions['buy_signals']]
sell_times = t[trading_decisions['sell_signals']]

if len(buy_times) > 0:
    ax7.scatter(buy_times, final_signal['signal'][trading_decisions['buy_signals']], 
               c='green', marker='^', s=50, label='Buy', zorder=5)
if len(sell_times) > 0:
    ax7.scatter(sell_times, final_signal['signal'][trading_decisions['sell_signals']], 
               c='red', marker='v', s=50, label='Sell', zorder=5)

ax7.set_title('Final Trading Signal')
ax7.set_xlabel('Time')
ax7.set_ylabel('Signal')
ax7.legend()
ax7.grid(True, alpha=0.3)

# 6. Cross-scale coherence matrix (bottom row, left)
ax8 = fig.add_subplot(gs[3, :2])
coherence_matrix = coherence_analysis['cross_scale_coherence']
im3 = ax8.imshow(coherence_matrix, cmap='viridis', aspect='auto')
ax8.set_title('Cross-Scale Coherence Matrix')
ax8.set_xlabel('Scale Index')
ax8.set_ylabel('Scale Index')
plt.colorbar(im3, ax=ax8, label='Coherence')

# 7. Performance summary (bottom row, right)
ax9 = fig.add_subplot(gs[3, 2:])
ax9.axis('off')

# Summary statistics
summary_text = f"""
MULTI-TIMEFRAME ANALYSIS SUMMARY

Scale Distribution:
• Micro events: {np.sum(pattern_analysis['micro_events'])}
• Meso events: {np.sum(pattern_analysis['meso_events'])}
• Macro events: {np.sum(pattern_analysis['macro_events'])}

Energy Transfer:
• Upscale: {energy_transfer['upscale_transfer']:.4f}
• Downscale: {energy_transfer['downscale_transfer']:.4f}
• Net transfer: {energy_transfer['net_transfer']:.4f}

Trading Signals:
• Buy signals: {np.sum(trading_decisions['buy_signals'])}
• Sell signals: {np.sum(trading_decisions['sell_signals'])}
• Hold periods: {np.sum(trading_decisions['hold_signals'])}

Adaptive Weights:
• Micro: {final_signal['adaptive_weights']['micro']:.3f}
• Meso: {final_signal['adaptive_weights']['meso']:.3f}
• Macro: {final_signal['adaptive_weights']['macro']:.3f}

Signal Quality:
• Mean confidence: {np.mean(final_signal['confidence']):.3f}
• Signal range: [{np.min(final_signal['signal']):.3f}, {np.max(final_signal['signal']):.3f}]
• Position sizing: {np.mean(trading_decisions['position_size']):.3f} avg

TRADING IMPLICATIONS:
• {'Strong trend following regime' if energy_transfer['upscale_transfer'] > 0.01 else 'Choppy/ranging market' if abs(energy_transfer['net_transfer']) < 0.005 else 'Mixed regime with volatility clustering'}
• {'High confidence signals' if np.mean(final_signal['confidence']) > 0.5 else 'Moderate confidence' if np.mean(final_signal['confidence']) > 0.3 else 'Low confidence - reduce position sizes'}
"""

ax9.text(0.05, 0.95, summary_text, transform=ax9.transAxes, 
         fontsize=11, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("MULTI-TIMEFRAME FLUID DYNAMICS ANALYSIS COMPLETE")
print("="*70)
print(f"Analysis processed {len(market_data['time'])} time points across {len(cwt_result['periods'])} scales")
print(f"Energy cascade direction: {'Upscale dominant' if energy_transfer['upscale_transfer'] > abs(energy_transfer['downscale_transfer']) else 'Downscale dominant'}")
print(f"Signal generation resulted in {np.sum(trading_decisions['buy_signals']) + np.sum(trading_decisions['sell_signals'])} trading opportunities")
print(f"Average signal confidence: {np.mean(final_signal['confidence']):.1%}")
print("\nThis analysis enables:")
print("• Scale-aware pattern recognition")
print("• Energy cascade-based regime detection")
print("• Multi-timeframe signal synthesis")
print("• Adaptive position sizing")
print("="*70)